In [6]:
!pip install scikit-survival

!pip install shap
!pip install pydantic pandas_profiling
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install scikit-survival
!pip install scikit-learn
!pip install xgboost
!pip install shap


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
# from google.colab import drive
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
# from sksurv.meta import EnsembleSelection
# from sksurv.svm import FastSurvivalSVM, FastKernelSurvivalSVM
# from sksurv.tree import SurvivalTree
# from sksurv.ensemble import GradientBoostingSurvivalAnalysis
# from sksurv.linear_model import CoxPHSurvivalAnalysis, CoxnetSurvivalAnalysis
# from sksurv.metrics import concordance_index_censored
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.model_selection import RandomizedSearchCV, train_test_split, StratifiedKFold, RepeatedStratifiedKFold, cross_val_score
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif, VarianceThreshold, RFECV
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.ensemble import GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn import set_config
# import xgboost as xgb
# import shap
import pickle
import time
from copy import deepcopy
from functools import cache
import warnings
warnings.filterwarnings("ignore")
randomState = 42 # tip of the cap to Douglas Adams
# set_config(transform_output="pandas")
Verbosity = 0

In [3]:
# !install sksurv


In [4]:
#### LOAD DATA

In [5]:
# CLASSIFICATION DATA
# data_path = 'C:\\Users\\T00733937\\Downloads\\All_TCGA_Train.csv'
new_labels = 'C:\\Users\\T00758722\\Downloads\\Matrix_WHO2021.csv'
new_catagories = pd.read_csv(new_labels)


In [6]:
# CLINICAL DATA
main_data = 'C:\\Users\\T00758722\\Downloads\\2019_TCGA-CDR-SupplementalTableS1.xlsx'
main_data_df = pd.read_excel(main_data)

In [7]:
new_catagories.head()

,Patient_ID,TCGA-histological.type,classification.2021_complete.labels,classification.2021_simplified.labels
0,TCGA-02-0001,untreated primary (de novo) gbm,glioblastoma_IDHwt,glioblastoma
1,TCGA-02-0003,untreated primary (de novo) gbm,glioblastoma_IDHwt,glioblastoma
2,TCGA-02-0004,untreated primary (de novo) gbm,unclassified,unclassified
3,TCGA-02-0006,untreated primary (de novo) gbm,glioblastoma_IDHwt,glioblastoma
4,TCGA-02-0007,treated primary gbm,glioblastoma_IDHwt,glioblastoma


In [8]:
main_data_df.head()

,Unnamed: 0,bcr_patient_barcode,type,age_at_initial_pathologic_diagnosis,gender,race,ajcc_pathologic_tumor_stage,clinical_stage,histological_type,histological_grade,...,residual_tumor,OS,OS.time,DSS,DSS.time,PFI,PFI.time,DFI,DFI.time,Redaction
0,1,TCGA-OR-A5J1,ACC,58.0,MALE,WHITE,Stage II,[Not Applicable],Adrenocortical carcinoma- Usual Type,[Not Available],...,NaN,1.0,1355.0,1.0,1355.0,1.0,754.0,1.0,754.0,NaN
1,2,TCGA-OR-A5J2,ACC,44.0,FEMALE,WHITE,Stage IV,[Not Applicable],Adrenocortical carcinoma- Usual Type,[Not Available],...,NaN,1.0,1677.0,1.0,1677.0,1.0,289.0,NaN,NaN,NaN
2,3,TCGA-OR-A5J3,ACC,23.0,FEMALE,WHITE,Stage III,[Not Applicable],Adrenocortical carcinoma- Usual Type,[Not Available],...,NaN,0.0,2091.0,0.0,2091.0,1.0,53.0,1.0,53.0,NaN
3,4,TCGA-OR-A5J4,ACC,23.0,FEMALE,WHITE,Stage IV,[Not Applicable],Adrenocortical carcinoma- Usual Type,[Not Available],...,NaN,1.0,423.0,1.0,423.0,1.0,126.0,NaN,NaN,NaN
4,5,TCGA-OR-A5J5,ACC,30.0,MALE,WHITE,Stage III,[Not Applicable],Adrenocortical carcinoma- Usual Type,[Not Available],...,NaN,1.0,365.0,1.0,365.0,1.0,50.0,NaN,NaN,NaN


In [9]:
main_data_df.rename(columns={'bcr_patient_barcode': 'Patient_ID'}, inplace=True)


In [10]:
# Filter rows where type is GBM or LGG
filtered_data_df = main_data_df[main_data_df['type'].isin(['GBM', 'LGG'])]

In [11]:
filtered_data_df.shape

(1111, 34)

In [12]:
# Check data types of all columns
column_types = filtered_data_df.dtypes

# Print the data types
print(column_types)

Unnamed: 0                               int64
Patient_ID                              object
type                                    object
age_at_initial_pathologic_diagnosis    float64
gender                                  object
race                                    object
ajcc_pathologic_tumor_stage             object
clinical_stage                          object
histological_type                       object
histological_grade                      object
initial_pathologic_dx_year             float64
menopause_status                        object
birth_days_to                          float64
vital_status                            object
tumor_status                            object
last_contact_days_to                   float64
death_days_to                          float64
cause_of_death                          object
new_tumor_event_type                    object
new_tumor_event_site                    object
new_tumor_event_site_other              object
new_tumor_eve

In [13]:
#Empty column
column_data = filtered_data_df["margin_status"]
print(column_data)

2645    NaN
2646    NaN
2647    NaN
2648    NaN
2649    NaN
       ... 
5420    NaN
5421    NaN
5422    NaN
5423    NaN
5424    NaN
Name: margin_status, Length: 1111, dtype: object


In [14]:
#Clinical+classification info
main_data_merged = pd.merge(filtered_data_df, new_catagories, on='Patient_ID', how='left')


In [15]:
main_data_merged.head()

,Unnamed: 0,Patient_ID,type,age_at_initial_pathologic_diagnosis,gender,race,ajcc_pathologic_tumor_stage,clinical_stage,histological_type,histological_grade,...,DSS,DSS.time,PFI,PFI.time,DFI,DFI.time,Redaction,TCGA-histological.type,classification.2021_complete.labels,classification.2021_simplified.labels
0,2646,TCGA-02-0001,GBM,44.0,FEMALE,WHITE,[Not Available],[Not Applicable],Untreated primary (de novo) GBM,[Not Available],...,1.0,358.0,1.0,137.0,NaN,NaN,NaN,untreated primary (de novo) gbm,glioblastoma_IDHwt,glioblastoma
1,2647,TCGA-02-0003,GBM,50.0,MALE,WHITE,[Not Available],[Not Applicable],Untreated primary (de novo) GBM,[Not Available],...,1.0,144.0,1.0,40.0,NaN,NaN,NaN,untreated primary (de novo) gbm,glioblastoma_IDHwt,glioblastoma
2,2648,TCGA-02-0004,GBM,59.0,MALE,WHITE,[Not Available],[Not Applicable],Untreated primary (de novo) GBM,[Not Available],...,1.0,345.0,1.0,314.0,NaN,NaN,NaN,untreated primary (de novo) gbm,unclassified,unclassified
3,2649,TCGA-02-0006,GBM,56.0,FEMALE,WHITE,[Not Available],[Not Applicable],Untreated primary (de novo) GBM,[Not Available],...,1.0,558.0,1.0,302.0,NaN,NaN,NaN,untreated primary (de novo) gbm,glioblastoma_IDHwt,glioblastoma
4,2650,TCGA-02-0007,GBM,40.0,FEMALE,WHITE,[Not Available],[Not Applicable],Treated primary GBM,[Not Available],...,1.0,705.0,1.0,518.0,NaN,NaN,NaN,treated primary gbm,glioblastoma_IDHwt,glioblastoma


In [16]:
# main_data_merged.to_csv("C:\\Users\\T00733937\\Documents\\main_data_merged.csv")

In [17]:
# from flask import Flask, send_file
# import pandas as pd
# import io

# # Convert filtered_data_df to CSV in memory
# csv_output = io.StringIO()
# main_data_merged.to_csv(csv_output, index=False)
# csv_output.seek(0)  # Move cursor to beginning of StringIO object

# # Flask application setup
# app = Flask(__name__)

# @app.route('/download_csv')
# def download_csv():
#     return send_file(
#         csv_output,
#         mimetype='text/csv',
#         as_attachment=True,
#         download_name='filtered_data.csv'
#     )

# if __name__ == '__main__':
#     app.run(debug=True)

In [18]:
# filtered_data_df.to_csv("filtered_data", index=False)
# # Flask application setup
# app = Flask(__name__)

# @app.route('/download_csv')
# def download_csv():
#     return send_file(
#         csv_output,
#         mimetype='text/csv',
#         as_attachment=True,
#         download_name='filtered_data.csv'
#     )

# if __name__ == '__main__':
#     app.run(debug=True)

In [19]:
# Specify the column of interest
column_name = 'classification.2021_simplified.labels'

# Count NaNs in the column
nan_count = main_data_merged[column_name].isna().sum()

# Count empty strings in the column (assuming empty strings)
empty_count = (main_data_merged[column_name] == '').sum()

# Count total number of rows
total_rows = len(main_data_merged)

# Count rows where the column is entirely NaN or empty string
na_or_empty_rows = main_data_merged[main_data_merged[column_name].isna() | (main_data_merged[column_name] == '')].shape[0]

# Print the results
print(f"Number of NaNs in '{column_name}': {nan_count}")
print(f"Number of empty strings in '{column_name}': {empty_count}")
print(f"Total number of rows: {total_rows}")
print(f"Number of rows with NaNs or empty strings in '{column_name}': {na_or_empty_rows}")

Number of NaNs in 'classification.2021_simplified.labels': 1
Number of empty strings in 'classification.2021_simplified.labels': 0
Total number of rows: 1111
Number of rows with NaNs or empty strings in 'classification.2021_simplified.labels': 1


In [20]:
###### LOAD METHYLATION DATA

# data_path = 'C:\\Users\\T00733937\\Downloads\\All_TCGA_Train.csv'
gbm_path = 'C:\\Users\\T00758722\\Downloads\\GBM_450K_Filtered_X-Y,SNPs,Non-overlap850K.csv'
gbm_data = pd.read_csv(gbm_path)
lgg_path = 'C:\\Users\\T00758722\\Downloads\\LGG-450K_Filtered_X-Y,SNPs,Non-overlap850K.csv'
lgg_data = pd.read_csv(lgg_path)

In [21]:
gbm_data.head()

,Unnamed: 0,TCGA-14-1043,TCGA-06-0125,TCGA-19-1389,TCGA-14-1395,TCGA-14-1402,TCGA-26-1442,TCGA-15-1444,TCGA-14-1450,TCGA-06-0152,...,TCGA-RR-A6KB,TCGA-RR-A6KC,TCGA-06-A6S0,TCGA-06-A6S1,TCGA-06-A7TK,TCGA-06-A7TL,TCGA-26-A7UX,TCGA-4W-AA9R,TCGA-4W-AA9S,TCGA-4W-AA9T
0,cg13869341,0.839668,0.887214,0.878038,0.857886,0.881700,0.851489,0.885795,0.906938,0.843728,...,0.919237,0.945097,0.910316,0.918181,0.925097,0.836280,0.852900,0.849698,0.851568,0.881214
1,cg24669183,0.622626,0.557666,0.821218,0.765783,0.755165,0.727787,0.682416,0.848990,0.617183,...,0.642642,0.838042,0.846426,0.680537,0.785081,0.871515,0.825222,0.657936,0.770009,0.859177
2,cg15560884,0.724557,0.709998,0.665864,0.677472,0.691028,0.790580,0.795775,0.730360,0.658813,...,0.680196,0.778355,0.704736,0.780342,0.624428,0.700333,0.480670,0.535313,0.668259,0.583731
3,cg01014490,0.013253,0.016885,0.017920,0.008228,0.021444,0.011233,0.011740,0.012311,0.014551,...,0.014228,0.013987,0.013409,0.013271,0.017785,0.035610,0.032341,0.020090,0.015112,0.016414
4,cg17505339,0.954777,0.909813,0.925042,0.952425,0.938924,0.909230,0.920223,0.946707,0.939457,...,0.923919,0.922130,0.929961,0.891294,0.889130,0.877647,0.921521,0.837631,0.908364,0.893055


In [22]:
# Change columns to rows
gbm_trns=gbm_data.transpose()

In [23]:
new_header = gbm_trns.iloc[0] #grab the first row for the header
print(new_header)
gbm_trns.columns = new_header #set the header row as the df header

gbm_trns_final=gbm_trns[1:]
gbm_trns_final.shape

0         cg13869341
1         cg24669183
2         cg15560884
3         cg01014490
4         cg17505339
             ...    
403949    cg05615487
403950    cg22122449
403951    cg08423507
403952    cg19565306
403953    cg09226288
Name: Unnamed: 0, Length: 403954, dtype: object


(140, 403954)

In [24]:
lgg_data.head()
lgg_trns=lgg_data.transpose()
new_header = lgg_trns.iloc[0] #grab the first row for the header
print(new_header)
lgg_trns.columns = new_header #set the header row as the df header

lgg_trns_final=lgg_trns[1:]
lgg_trns_final.shape

0         cg13869341
1         cg24669183
2         cg15560884
3         cg01014490
4         cg17505339
             ...    
403949    cg05615487
403950    cg22122449
403951    cg08423507
403952    cg19565306
403953    cg09226288
Name: Unnamed: 0, Length: 403954, dtype: object


(516, 403954)

In [25]:
gbm_trns_final.head()

Unnamed: 0,cg13869341,cg24669183,cg15560884,cg01014490,cg17505339,cg11954957,cg16736630,cg05898754,cg03128332,cg16619049,...,cg09635994,cg19004771,cg20569369,cg26034629,cg25232725,cg05615487,cg22122449,cg08423507,cg19565306,cg09226288
TCGA-14-1043,0.839668,0.622626,0.724557,0.013253,0.954777,0.77868,0.937495,0.574408,0.256935,0.346032,...,0.944636,0.915037,0.79652,0.867592,0.88258,0.86959,0.82853,0.891635,0.015408,0.770533
TCGA-06-0125,0.887214,0.557666,0.709998,0.016885,0.909813,0.859712,0.8852,0.341588,0.088496,0.115528,...,0.949723,0.897736,0.827256,0.867966,0.814527,0.519711,0.516864,0.353613,0.018237,0.747907
TCGA-19-1389,0.878038,0.821218,0.665864,0.01792,0.925042,0.815867,0.818661,0.365594,0.115413,0.206443,...,0.935232,0.836684,0.810572,0.894819,0.67194,0.845,0.840595,0.735885,0.019254,0.723458
TCGA-14-1395,0.857886,0.765783,0.677472,0.008228,0.952425,0.691414,0.915337,0.337019,0.258516,0.078493,...,0.941118,0.89064,0.507454,0.775865,0.833592,0.744148,0.496444,0.181154,0.017205,0.803489
TCGA-14-1402,0.8817,0.755165,0.691028,0.021444,0.938924,0.86719,0.874905,0.684556,0.113998,0.370365,...,0.936734,0.905482,0.86603,0.896591,0.859173,0.930482,0.880097,0.953423,0.015289,0.681242


In [26]:
lgg_trns_final.head()

Unnamed: 0,cg13869341,cg24669183,cg15560884,cg01014490,cg17505339,cg11954957,cg16736630,cg05898754,cg03128332,cg16619049,...,cg09635994,cg19004771,cg20569369,cg26034629,cg25232725,cg05615487,cg22122449,cg08423507,cg19565306,cg09226288
TCGA-CS-4938,0.936116,0.898359,0.845336,0.015968,0.946454,0.886434,0.892665,0.809288,0.035519,0.799344,...,0.968511,0.946421,0.890017,0.907129,0.891813,0.946324,0.912266,0.980936,0.021703,0.823637
TCGA-CS-4941,0.793768,0.815238,0.619141,0.012017,0.936335,0.831279,0.894495,0.269462,0.228493,0.165512,...,0.878442,0.759526,0.645649,0.854135,0.78282,0.620469,0.75128,0.82387,0.015897,0.798146
TCGA-CS-4942,0.933341,0.902362,0.658365,0.015339,0.942698,0.803588,0.897181,0.519737,0.202468,0.777195,...,0.831774,0.944349,0.903782,0.922507,0.874409,0.940929,0.908363,0.9711,0.014927,0.724329
TCGA-CS-4943,0.911212,0.846652,0.618483,0.017999,0.94731,0.85072,0.922721,0.504014,0.396098,0.71867,...,0.75166,0.948531,0.866245,0.933678,0.891107,0.935906,0.955214,0.964724,0.017563,0.727105
TCGA-CS-4944,0.910449,0.886066,0.753085,0.013121,0.927629,0.829942,0.943098,0.621303,0.248887,0.684886,...,0.900367,0.940183,0.908439,0.935673,0.896156,0.927853,0.920689,0.969393,0.016242,0.772883


In [27]:
### MERGE METHYLATION DATA (WHO 2021 CLASSIFICATION)

In [28]:
merged_df = pd.concat([gbm_trns_final, lgg_trns_final], ignore_index=False)


In [29]:
merged_df.shape

(656, 403954)

In [30]:
# merged_df.to_csv("C:\\Users\\T00733937\\Documents\\gene_data_merged.csv")

In [31]:
merged_df.rename(columns={'Index': 'Patient_ID'}, inplace=True)


In [32]:
merged_df.head()

Unnamed: 0,cg13869341,cg24669183,cg15560884,cg01014490,cg17505339,cg11954957,cg16736630,cg05898754,cg03128332,cg16619049,...,cg09635994,cg19004771,cg20569369,cg26034629,cg25232725,cg05615487,cg22122449,cg08423507,cg19565306,cg09226288
TCGA-14-1043,0.839668,0.622626,0.724557,0.013253,0.954777,0.77868,0.937495,0.574408,0.256935,0.346032,...,0.944636,0.915037,0.79652,0.867592,0.88258,0.86959,0.82853,0.891635,0.015408,0.770533
TCGA-06-0125,0.887214,0.557666,0.709998,0.016885,0.909813,0.859712,0.8852,0.341588,0.088496,0.115528,...,0.949723,0.897736,0.827256,0.867966,0.814527,0.519711,0.516864,0.353613,0.018237,0.747907
TCGA-19-1389,0.878038,0.821218,0.665864,0.01792,0.925042,0.815867,0.818661,0.365594,0.115413,0.206443,...,0.935232,0.836684,0.810572,0.894819,0.67194,0.845,0.840595,0.735885,0.019254,0.723458
TCGA-14-1395,0.857886,0.765783,0.677472,0.008228,0.952425,0.691414,0.915337,0.337019,0.258516,0.078493,...,0.941118,0.89064,0.507454,0.775865,0.833592,0.744148,0.496444,0.181154,0.017205,0.803489
TCGA-14-1402,0.8817,0.755165,0.691028,0.021444,0.938924,0.86719,0.874905,0.684556,0.113998,0.370365,...,0.936734,0.905482,0.86603,0.896591,0.859173,0.930482,0.880097,0.953423,0.015289,0.681242


In [33]:
# # Merge based on different column names
# final_main_data_merged = pd.merge(main_data_merged, merged_df,
#                                   left_on='Patient_ID', right_index=True,
#                                   how='left')

In [34]:
### MERGE CLINICAL AND METHYLATION
# Merge based on different column names
final_main_data_merged = pd.merge(merged_df, main_data_merged,
                                  left_index=True, right_on='Patient_ID',
                                  how='left')

In [35]:
final_main_data_merged.shape

(656, 403991)

In [36]:
# !pip install pickel
import pickle

# Save DataFrame to binary (pickle) format
with open('data.pickle', 'wb') as f:
    pickle.dump(final_main_data_merged, f)

In [37]:
# os.getcwd()

In [38]:
# Load DataFrame from binary (pickle) format
with open('data.pickle', 'rb') as f:
    glioma_data = pickle.load(f)

# Display the loaded DataFrame
glioma_data.head()

,cg13869341,cg24669183,cg15560884,cg01014490,cg17505339,cg11954957,cg16736630,cg05898754,cg03128332,cg16619049,...,DSS,DSS.time,PFI,PFI.time,DFI,DFI.time,Redaction,TCGA-histological.type,classification.2021_complete.labels,classification.2021_simplified.labels
369.0,0.839668,0.622626,0.724557,0.013253,0.954777,0.77868,0.937495,0.574408,0.256935,0.346032,...,1.0,24.0,1.0,24.0,NaN,NaN,NaN,untreated primary (de novo) gbm,glioblastoma_IDHwt,glioblastoma
96.0,0.887214,0.557666,0.709998,0.016885,0.909813,0.859712,0.8852,0.341588,0.088496,0.115528,...,1.0,1448.0,1.0,797.0,NaN,NaN,NaN,untreated primary (de novo) gbm,glioblastoma_IDHwt,glioblastoma
423.0,0.878038,0.821218,0.665864,0.01792,0.925042,0.815867,0.818661,0.365594,0.115413,0.206443,...,1.0,141.0,1.0,81.0,NaN,NaN,NaN,untreated primary (de novo) gbm,glioblastoma_IDHwt,glioblastoma
370.0,0.857886,0.765783,0.677472,0.008228,0.952425,0.691414,0.915337,0.337019,0.258516,0.078493,...,1.0,42.0,1.0,42.0,NaN,NaN,NaN,untreated primary (de novo) gbm,glioblastoma_IDHwt,glioblastoma
373.0,0.8817,0.755165,0.691028,0.021444,0.938924,0.86719,0.874905,0.684556,0.113998,0.370365,...,1.0,975.0,1.0,333.0,NaN,NaN,NaN,untreated primary (de novo) gbm,unclassified,unclassified


In [39]:
glioma_data.shape

(656, 403991)

In [40]:
# Determine columns to drop based on the criteria
columns_to_drop = []
for col in glioma_data.columns:
    # Count total number of rows in the column
    total_rows = len(glioma_data)
    
    # Count NaN, empty, or specific strings like 'NaN' or '/[Not Available/]'
    empty_or_nan_count = glioma_data[col].apply(lambda x: pd.isna(x) or x == '' or x == 'NaN' or x == '[Not Available]').sum()
    
    # If more than 50 rows are empty or NaN, mark column for dropping
    if empty_or_nan_count >= 50:
        columns_to_drop.append(col)

# Drop identified columns from the DataFrame
glioma_data_filtered = glioma_data.drop(columns=columns_to_drop)
# Print the filtered DataFrame


In [41]:
print(glioma_data_filtered.head())
# print(glioma_data_filtered.shape)

      cg13869341 cg24669183 cg15560884 cg01014490 cg17505339 cg11954957  \
369.0   0.839668   0.622626   0.724557   0.013253   0.954777    0.77868   
96.0    0.887214   0.557666   0.709998   0.016885   0.909813   0.859712   
423.0   0.878038   0.821218   0.665864    0.01792   0.925042   0.815867   
370.0   0.857886   0.765783   0.677472   0.008228   0.952425   0.691414   
373.0     0.8817   0.755165   0.691028   0.021444   0.938924    0.86719   

      cg16736630 cg05898754 cg03128332 cg16619049  ... tumor_status   OS  \
369.0   0.937495   0.574408   0.256935   0.346032  ...   WITH TUMOR  1.0   
96.0      0.8852   0.341588   0.088496   0.115528  ...   WITH TUMOR  1.0   
423.0   0.818661   0.365594   0.115413   0.206443  ...   WITH TUMOR  1.0   
370.0   0.915337   0.337019   0.258516   0.078493  ...   WITH TUMOR  1.0   
373.0   0.874905   0.684556   0.113998   0.370365  ...   WITH TUMOR  1.0   

      OS.time  DSS DSS.time  PFI PFI.time           TCGA-histological.type  \
369.0    24.0 

In [42]:
# glioma_data_filtered[target_col].head()
# glioma_data_filtered_2 = glioma_data_filtered.drop(columns="gender")

In [43]:
from sklearn.preprocessing import LabelEncoder
data_frame = glioma_data_filtered
# Load your data
# data_frame = pd.read_csv('your_data.csv')

def clean_data(data_frame):
    # Drop rows with any NaN values
    data_frame_cleaned = data_frame.dropna()

    # Drop rows with any empty cells (if cells are empty strings)
    data_frame_cleaned = data_frame_cleaned.replace(r'^\s*$', pd.NA, regex=True).dropna()

    return data_frame_cleaned

def encode_target(data_frame, target_col):
    # Initialize the LabelEncoder
    le = LabelEncoder()

    # Fit and transform the target column
    data_frame[target_col] = le.fit_transform(data_frame[target_col])

    return data_frame, le

# Clean the data
cleaned_data_frame = clean_data(data_frame)

# Encode the target variable
target_col =  'classification.2021_simplified.labels' # Replace with your actual target column name
encoded_data_frame, label_encoder = encode_target(cleaned_data_frame, target_col)





# Display the first few rows of the cleaned and encoded DataFrame
print(encoded_data_frame.head())
print(encoded_data_frame.shape)

# If you want to see the mapping of classes to numbers
print("Class mapping:")
for index, class_ in enumerate(label_encoder.classes_):
    print(f'{class_}: {index}')


       cg13869341  cg24669183  cg15560884  cg01014490  cg17505339  cg11954957  \
369.0    0.839668    0.622626    0.724557    0.013253    0.954777    0.778680   
96.0     0.887214    0.557666    0.709998    0.016885    0.909813    0.859712   
423.0    0.878038    0.821218    0.665864    0.017920    0.925042    0.815867   
370.0    0.857886    0.765783    0.677472    0.008228    0.952425    0.691414   
373.0    0.881700    0.755165    0.691028    0.021444    0.938924    0.867190   

       cg16736630  cg05898754  cg03128332  cg16619049  ...  tumor_status   OS  \
369.0    0.937495    0.574408    0.256935    0.346032  ...    WITH TUMOR  1.0   
96.0     0.885200    0.341588    0.088496    0.115528  ...    WITH TUMOR  1.0   
423.0    0.818661    0.365594    0.115413    0.206443  ...    WITH TUMOR  1.0   
370.0    0.915337    0.337019    0.258516    0.078493  ...    WITH TUMOR  1.0   
373.0    0.874905    0.684556    0.113998    0.370365  ...    WITH TUMOR  1.0   

       OS.time  DSS  DSS.t

In [44]:
print(encoded_data_frame.head())


       cg13869341  cg24669183  cg15560884  cg01014490  cg17505339  cg11954957  \
369.0    0.839668    0.622626    0.724557    0.013253    0.954777    0.778680   
96.0     0.887214    0.557666    0.709998    0.016885    0.909813    0.859712   
423.0    0.878038    0.821218    0.665864    0.017920    0.925042    0.815867   
370.0    0.857886    0.765783    0.677472    0.008228    0.952425    0.691414   
373.0    0.881700    0.755165    0.691028    0.021444    0.938924    0.867190   

       cg16736630  cg05898754  cg03128332  cg16619049  ...  tumor_status   OS  \
369.0    0.937495    0.574408    0.256935    0.346032  ...    WITH TUMOR  1.0   
96.0     0.885200    0.341588    0.088496    0.115528  ...    WITH TUMOR  1.0   
423.0    0.818661    0.365594    0.115413    0.206443  ...    WITH TUMOR  1.0   
370.0    0.915337    0.337019    0.258516    0.078493  ...    WITH TUMOR  1.0   
373.0    0.874905    0.684556    0.113998    0.370365  ...    WITH TUMOR  1.0   

       OS.time  DSS  DSS.t

In [45]:
unclassified_count = encoded_data_frame['classification.2021_simplified.labels'].value_counts().get(3, 0)


In [46]:
print(unclassified_count)

33


In [47]:
cleaned_glioma_data = encoded_data_frame[encoded_data_frame['classification.2021_simplified.labels'] != 3]


In [48]:
cleaned_glioma_data.head()

,cg13869341,cg24669183,cg15560884,cg01014490,cg17505339,cg11954957,cg16736630,cg05898754,cg03128332,cg16619049,...,tumor_status,OS,OS.time,DSS,DSS.time,PFI,PFI.time,TCGA-histological.type,classification.2021_complete.labels,classification.2021_simplified.labels
369.0,0.839668,0.622626,0.724557,0.013253,0.954777,0.778680,0.937495,0.574408,0.256935,0.346032,...,WITH TUMOR,1.0,24.0,1.0,24.0,1.0,24.0,untreated primary (de novo) gbm,glioblastoma_IDHwt,1
96.0,0.887214,0.557666,0.709998,0.016885,0.909813,0.859712,0.885200,0.341588,0.088496,0.115528,...,WITH TUMOR,1.0,1448.0,1.0,1448.0,1.0,797.0,untreated primary (de novo) gbm,glioblastoma_IDHwt,1
423.0,0.878038,0.821218,0.665864,0.017920,0.925042,0.815867,0.818661,0.365594,0.115413,0.206443,...,WITH TUMOR,1.0,141.0,1.0,141.0,1.0,81.0,untreated primary (de novo) gbm,glioblastoma_IDHwt,1
370.0,0.857886,0.765783,0.677472,0.008228,0.952425,0.691414,0.915337,0.337019,0.258516,0.078493,...,WITH TUMOR,1.0,42.0,1.0,42.0,1.0,42.0,untreated primary (de novo) gbm,glioblastoma_IDHwt,1
459.0,0.851489,0.727787,0.790580,0.011233,0.909230,0.632709,0.839941,0.651802,0.071423,0.607906,...,WITH TUMOR,0.0,953.0,0.0,953.0,0.0,953.0,untreated primary (de novo) gbm,astrocytoma_IDHmut,0


In [49]:
cleaned_glioma_data.shape

(578, 403974)

In [50]:
# Count occurrences of '[not available]' in each column
not_available_counts = cleaned_glioma_data.apply(lambda col: col[col == '[Not Available]'].count())

# Determine columns to drop based on the count
# Delete columns with more than 5 N/A
columns_to_drop = not_available_counts[not_available_counts > 5].index.tolist()

# Drop identified columns from the DataFrame
filtered_glioma_data = cleaned_glioma_data.drop(columns=columns_to_drop)

# Print the filtered DataFrame


In [51]:
print(filtered_glioma_data.head())

       cg13869341  cg24669183  cg15560884  cg01014490  cg17505339  cg11954957  \
369.0    0.839668    0.622626    0.724557    0.013253    0.954777    0.778680   
96.0     0.887214    0.557666    0.709998    0.016885    0.909813    0.859712   
423.0    0.878038    0.821218    0.665864    0.017920    0.925042    0.815867   
370.0    0.857886    0.765783    0.677472    0.008228    0.952425    0.691414   
459.0    0.851489    0.727787    0.790580    0.011233    0.909230    0.632709   

       cg16736630  cg05898754  cg03128332  cg16619049  ...  tumor_status   OS  \
369.0    0.937495    0.574408    0.256935    0.346032  ...    WITH TUMOR  1.0   
96.0     0.885200    0.341588    0.088496    0.115528  ...    WITH TUMOR  1.0   
423.0    0.818661    0.365594    0.115413    0.206443  ...    WITH TUMOR  1.0   
370.0    0.915337    0.337019    0.258516    0.078493  ...    WITH TUMOR  1.0   
459.0    0.839941    0.651802    0.071423    0.607906  ...    WITH TUMOR  0.0   

       OS.time  DSS  DSS.t

In [52]:
# Check data types of last 34 columns
last_34_columns_types = filtered_glioma_data.iloc[:, -34:].dtypes

# Print the data types of last 34 columns
print(last_34_columns_types)

cg17074573                               float64
cg03403991                               float64
cg12502079                               float64
cg06422471                               float64
cg19324023                               float64
cg09635994                               float64
cg19004771                               float64
cg20569369                               float64
cg26034629                               float64
cg25232725                               float64
cg05615487                               float64
cg22122449                               float64
cg08423507                               float64
cg19565306                               float64
cg09226288                               float64
Unnamed: 0                               float64
Patient_ID                                object
type                                      object
age_at_initial_pathologic_diagnosis      float64
gender                                    object
histological_type   

In [53]:
# Iterate over columns and check unique values for object dtype columns
for column_name in filtered_glioma_data.columns:
    if filtered_glioma_data[column_name].dtype == 'object':
        unique_values = filtered_glioma_data[column_name].unique()
        print(f"Unique values in {column_name}:")
        print(unique_values)
        print()

Unique values in Patient_ID:
['TCGA-14-1043' 'TCGA-06-0125' 'TCGA-19-1389' 'TCGA-14-1395'
 'TCGA-26-1442' 'TCGA-15-1444' 'TCGA-14-1450' 'TCGA-06-0152'
 'TCGA-06-0171' 'TCGA-06-1804' 'TCGA-06-1806' 'TCGA-06-0190'
 'TCGA-32-1979' 'TCGA-32-1980' 'TCGA-06-0210' 'TCGA-06-0211'
 'TCGA-06-0221' 'TCGA-19-4065' 'TCGA-76-4926' 'TCGA-76-4927'
 'TCGA-76-4928' 'TCGA-76-4929' 'TCGA-76-4931' 'TCGA-76-4932'
 'TCGA-76-4934' 'TCGA-76-4935' 'TCGA-26-5132' 'TCGA-26-5133'
 'TCGA-26-5134' 'TCGA-26-5136' 'TCGA-26-5139' 'TCGA-28-5204'
 'TCGA-28-5207' 'TCGA-28-5208' 'TCGA-28-5211' 'TCGA-28-5213'
 'TCGA-28-5214' 'TCGA-28-5215' 'TCGA-28-5216' 'TCGA-28-5219'
 'TCGA-28-5220' 'TCGA-32-5222' 'TCGA-12-5295' 'TCGA-12-5299'
 'TCGA-12-5301' 'TCGA-06-5408' 'TCGA-06-5410' 'TCGA-06-5411'
 'TCGA-06-5412' 'TCGA-06-5413' 'TCGA-06-5414' 'TCGA-06-5415'
 'TCGA-06-5416' 'TCGA-06-5418' 'TCGA-41-5651' 'TCGA-06-5856'
 'TCGA-06-5858' 'TCGA-06-5859' 'TCGA-87-5896' 'TCGA-81-5910'
 'TCGA-81-5911' 'TCGA-19-5947' 'TCGA-19-5950' 'TCGA-19-5

In [54]:
# # Save DataFrame to pickle file in current directory
# file_name = 'filtered_glioma_data.pkl'  # Specify the file name with .pkl extension
# filtered_glioma_data.to_pickle(file_name)

In [55]:
# Drop columns by name
columns_to_drop = [ 'classification.2021_complete.labels', 'TCGA-histological.type' , 'tumor_status','vital_status','histological_type','Patient_ID']  # List of column names to drop

filtered_glioma_data_2 = filtered_glioma_data.drop(columns=columns_to_drop)

In [56]:

# Iterate over columns and check unique values for object dtype columns
for column_name in filtered_glioma_data_2.columns:
    if filtered_glioma_data_2[column_name].dtype == 'object':
        unique_values = filtered_glioma_data_2[column_name].unique()
        print(f"Unique values in {column_name}:")
        print(unique_values)
        print()

Unique values in type:
['GBM' 'LGG']

Unique values in gender:
['MALE' 'FEMALE']



In [57]:
# Drop columns by name
columns_to_drop = ['type']  # List of column names to drop

filtered_glioma_data_2 = filtered_glioma_data_2.drop(columns=columns_to_drop)

In [58]:
# filtered_glioma_data_2 = filtered_glioma_data_2.drop(columns="classification.2021_complete.labels")

In [59]:
# Function to encode categorical columns using LabelEncoder
def encode_categorical_columns(data_frame, columns):
    encoded_data_frame = data_frame.copy()
    label_encoders = {}
    
    for column in columns:
        label_encoders[column] = LabelEncoder()
        encoded_data_frame[column] = label_encoders[column].fit_transform(encoded_data_frame[column])
    
    return encoded_data_frame, label_encoders

# List of columns to encode
columns_to_encode = ['gender']  # Replace with your actual columns

# Encode categorical columns
fianl_clean_data, label_encoders = encode_categorical_columns(filtered_glioma_data_2, columns_to_encode)


In [60]:
# Iterate over columns and check unique values for object dtype columns
for column_name in fianl_clean_data.columns:
    if fianl_clean_data[column_name].dtype == 'object':
        unique_values = fianl_clean_data[column_name].unique()
        print(f"Unique values in {column_name}:")
        print(unique_values)
        
        print()

In [61]:
fianl_clean_data.head()

,cg13869341,cg24669183,cg15560884,cg01014490,cg17505339,cg11954957,cg16736630,cg05898754,cg03128332,cg16619049,...,gender,initial_pathologic_dx_year,birth_days_to,OS,OS.time,DSS,DSS.time,PFI,PFI.time,classification.2021_simplified.labels
369.0,0.839668,0.622626,0.724557,0.013253,0.954777,0.778680,0.937495,0.574408,0.256935,0.346032,...,1,2004.0,-22392.0,1.0,24.0,1.0,24.0,1.0,24.0,1
96.0,0.887214,0.557666,0.709998,0.016885,0.909813,0.859712,0.885200,0.341588,0.088496,0.115528,...,0,2001.0,-23343.0,1.0,1448.0,1.0,1448.0,1.0,797.0,1
423.0,0.878038,0.821218,0.665864,0.017920,0.925042,0.815867,0.818661,0.365594,0.115413,0.206443,...,1,2008.0,-18659.0,1.0,141.0,1.0,141.0,1.0,81.0,1
370.0,0.857886,0.765783,0.677472,0.008228,0.952425,0.691414,0.915337,0.337019,0.258516,0.078493,...,1,2003.0,-19237.0,1.0,42.0,1.0,42.0,1.0,42.0,1
459.0,0.851489,0.727787,0.790580,0.011233,0.909230,0.632709,0.839941,0.651802,0.071423,0.607906,...,1,2008.0,-15950.0,0.0,953.0,0.0,953.0,0.0,953.0,0


In [62]:
fianl_clean_data.to_pickle('fianl_clean_data.pkl')